In [43]:
import pandas as pd
import os

In [64]:
def filename(file_dir):
    l = []
    for root, dirs, files in os.walk(file_dir):
        for file in files:
            if file.endswith('_info.csv'): 
                l.append(file)
    return l
            

In [69]:
path = os.getcwd()+'/data1/'
files = filename(path)
for f in files:
    df = pd.read_csv(path + f)
    df['date'] = pd.to_datetime(df.collect_time)
    df['date'] = df.date.dt.date

    charge = pd.DataFrame(df.groupby(['car_id','date'])['charge_usedtime','charged_soc'].sum())
    charge = charge.join(pd.DataFrame(df.groupby(['car_id','date'])['collect_time'].count()))
    charge = charge.reset_index()
    charge.to_csv(path+'charge.csv',mode='a+')

In [1]:
%matplotlib  inline 
from impala.dbapi  import connect 
from sqlalchemy import *  
from sqlalchemy.sql import select  
from sqlalchemy.schema import *  
import numpy as np
from sqlalchemy.orm import sessionmaker
import pandas as pd
import os  
from os.path import join, getsize  
from multiprocessing.dummy  import Pool 
import sys
reload(sys)
sys.setdefaultencoding("gbk")   ## window 下使用这种编码解决大部分问题 
conn=connect(host="10.10.36.237",port=21050,user="bjzx",password="Abc1234%")
cursor=conn.cursor()

In [24]:
def get_useinfo(e):
    lithum = pd.read_sql_query("select car_id,collect_time,receive_time,charge_status,current_soc from xxzx.his_lithium_can_data_v3 where dt='2016-12-01' and car_id=%s"%e,conn) 
    lithum['collect_time'] = pd.to_datetime(lithum.collect_time)
    lithum['receive_time'] = pd.to_datetime(lithum.receive_time)

    # 时间异常值处理
    lithum1 = lithum[(lithum.receive_time-lithum.collect_time).apply(lambda x: x.days>1)]
    lithum1.collect_time = lithum1.receive_time
    lithum2 = lithum[(lithum.receive_time-lithum.collect_time).apply(lambda x: x.days<=1)]
    lithum = pd.concat([lithum1,lithum2],axis=0)
    # 去重
    lithum.drop_duplicates(inplace=True)
    lithum.drop(['car_id','receive_time'],axis=1,inplace=True)

    lithum.sort_values('collect_time',inplace=True)
    lithum['label'] = lithum.charge_status-lithum.charge_status.shift(1)
    startcharge = lithum[abs(lithum.label)>1]  # 充电插拔
    del lithum,lithum1,lithum2
    if startcharge.shape[0] > 0:
        startcharge['charge_usedtime'] = (startcharge.collect_time.shift(-1) - startcharge.collect_time)
        startcharge['charged_soc'] = (startcharge.current_soc.shift(-1) - startcharge.current_soc)
        startcharge.dropna(inplace=True)
        used_info = startcharge[startcharge.label==3][['collect_time','charge_usedtime','current_soc','charged_soc']]
        used_info.rename(columns={'current_soc':'start_soc'},inplace=True)
        # 去掉异常值 一分钟充电量异常
        used_info.charge_usedtime = used_info.charge_usedtime.dt.seconds.apply(lambda x: x/60)
        used_info['unit_charge_soc'] = used_info.charged_soc/used_info.charge_usedtime
        used_info = used_info[(used_info.unit_charge_soc>0)&(used_info.unit_charge_soc<10)]
        used_info['car_id'] = e
        del startcharge
        return used_info


In [ ]:
lithumv3_id = pd.read_csv('data1/lithumv3.csv')
l = []
car_ids = lithumv3_id.lithumv3.tolist()

In [9]:
lithum_can = pd.DataFrame(columns=['collect_time','charge_usedtime','start_soc','charged_soc','unit_charge_soc','car_id'])
lithum_can

,collect_time,charge_usedtime,start_soc,charged_soc,unit_charge_soc,car_id


In [10]:
for i, e in enumerate(car_ids):
    try:
        lithum_can = pd.concat([lithum_can,get_useinfo(e)],axis=0)
    except:
        l.append(e)

In [14]:
lithum_can.to_csv('data1/lithumv3_info.csv',index=None)
lithum_can.car_id.nunique()

666